In [2]:
from ipyleaflet import Map, basemaps, basemap_to_tiles, Polygon, Circle, VectorTileLayer, Rectangle, LegendControl, AntPath
import numpy
from traitlets import Unicode, Dict
import pandas as pd




# set up for selecting colors for all the values. 
color = [1, 2, 3]

# Reading the globe data from the csv file 
globeTotalLiquid = pd.read_csv("/Users/arthivijayakumar/Documents/GitHub/precipitationEEETool/data/GLOBEMeasurementData2.csv", 
                    skiprows = [1], 
                    header = 0, 
                    usecols=[13, 14, 15, 16], 
                    dtype = { 'precipitations:liquid accumulation': float, 'precipitations:liquid accumulation flag' : str, 
                             'precipitations:snowfall accumulation' : float, 'precipitations:snowfall accumulation flag' : str}
                   )


globeData = pd.read_csv("/Users/arthivijayakumar/Documents/GitHub/precipitationEEETool/data/GLOBEMeasurementData2.csv", 
                    skiprows = [1], 
                    header = 0, 
                    usecols=[4, 5, 7], 
                    dtype = {' latitude': float, ' longitude': float, 'measured_on' : str}
                   )

# Scrubbing the data for missing flags in both the liquid and snowfall amounts
for i, j in globeTotalLiquid.iteritems(): 
    if i == "precipitations:liquid accumulation flag": 
        for i in range(len(j)): 
            if j[i] == 'missing': 
                ind = i
                globeTotalLiquid.at[ind,'precipitations:liquid accumulation'] = 0
            else:
                pass 
            

for i, j in globeTotalLiquid.iteritems(): 
    if i == "precipitations:snowfall accumulation flag": 
        for i in range(len(j)): 
            if j[i] == 'missing': 
                ind = i
                globeTotalLiquid.at[ind,'precipitations:snowfall accumulation'] = 0
            else:
                pass

globeTotalLiquid["precipitations:snowfall accumulation"] = globeTotalLiquid["precipitations:snowfall accumulation"].fillna(0)
globeTotalLiquid["precipitations:liquid accumulation"] = globeTotalLiquid["precipitations:liquid accumulation"].fillna(0)

# Changing the snow to its liquid equivalent
for i, j in globeTotalLiquid.iteritems():
    if i == "percipitations: snowfall accumulation": 
        for i in range(len(j)): 
            if j[i] != 0: 
                val = j[i]
                globeTotalLiquid.at[i, 'precipitations:snowfall accumulation'] = val / 13 
            else: 
                pass



totalLiquidCol = globeTotalLiquid["precipitations:snowfall accumulation"] + globeTotalLiquid["precipitations:liquid accumulation"]

globeTotalLiquid["total liquid accumulation"] = totalLiquidCol 

totalLiquid = globeTotalLiquid["total liquid accumulation"]

globeData["total liquid"] = totalLiquid 

# Reading the Daymet data from a csv file 

nasaData = pd.read_csv("/Users/arthivijayakumar/Documents/percipitationFinal/percipitationFinal-DAYMET-003-results.csv",  
                    header = 0, 
                    usecols = [2, 3, 4, 6], 
                    dtype = {'Latitude' : float, 'Longitude' : float, 'Date' : str, 'DAYMET_003_prcp' : float}
                      )

# Scrubbing the NASA data 
nasaData.loc[nasaData.DAYMET_003_prcp == -9999, 'DAYMET_003_prcp'] = 0

# The list of main Latitudes/Longitues in the area of interest
lat_long = [[42.030833, -70.9347], 
[42.030833, -71.272],
[42.030833, -71.6093],
[42.030833, -71.9466],
[42.257, -71.272],
[42.257, -70.9347],
[42.257, -71.6093],
[42.257, -71.9466],
[42.4831, -71.6093],
[42.4831, -70.9347],
[42.4831, -71.272],
[42.4831, -71.9466],
[42.7092, -71.9466],
[42.7092, -71.6093],
[42.7092, -71.272],
[42.7092, -70.9347],
]

sample = globeData.head(15)

movedItems = []
thisList = []
for i in globeData.itertuples(): 
    lat = float(i[1])
    long = float(i[2])
    thisList.append(lat)
    thisList.append(long)
    for j in lat_long: 
        latList = float(j[0])
        longList = float(j[1])
        if latList - 0.226122 <= lat <= latList + 0.226122 and longList - 0.3373 <= long <= longList + 0.3373: 
            y = list(i)
            y[1] = latList
            y[2] = longList
            del y[0]
            movedItems.append(y)


            
        
globeFinal = pd.DataFrame(numpy.array(movedItems), columns = ['Latitude', 'Longitude', "Date", "DAYMET_003_prcp"])


globeFinal[["Latitude", "Longitude", "DAYMET_003_prcp"]] = globeFinal[["Latitude", "Longitude", "DAYMET_003_prcp"]].apply(pd.to_numeric)
globeFinal.round(4)




# The merging of the two datasets 
finalData = pd.merge(globeFinal, nasaData, how='outer', on= ['Latitude', 'Longitude', "Date"])
finalData["DAYMET_003_prcp_x"] = finalData["DAYMET_003_prcp_x"].fillna(0)
finalData["DAYMET_003_prcp_y"] = finalData["DAYMET_003_prcp_y"].fillna(0)
            
sum_column = (finalData["DAYMET_003_prcp_x"] + finalData["DAYMET_003_prcp_y"]) * 0.5


finalData["sum_column"] = sum_column



year = finalData.Date.str.split('-').str[0]
finalData["year"] = year
finalData.loc[(finalData['Longitude'] < -71.27) & (finalData['Longitude'] > -71.3), 
    'Longitude'] = -71.272
def addingData(latitude, longitude, year): 
    year = str(year)
    return finalData.query("Latitude == " + latitude + "and Longitude == " + longitude + " and year == '" + year + "'")['sum_column'].sum()
userYear = str(input("What year would you like? Enter 2017, 2018, or 2019:     "))




longLat1 = addingData("42.030833", "-70.9347", userYear)
longLat2 = addingData("42.030833", "-71.272", userYear)
longLat3 = addingData("42.030833", "-71.6093", userYear)
longLat4 = addingData("42.030833", "-71.9466", userYear)
longLat5 = addingData("42.257", "-70.9347", userYear)
longLat6 = addingData("42.257", "-71.272", userYear)
longLat7 = addingData("42.257", "-71.6093", userYear)
longLat8 = addingData("42.257", "-71.9466", userYear)
longLat9 = addingData("42.4831", "-70.9347", userYear)
longLat10 = addingData("42.4831", "-71.272", userYear)
longLat11 = addingData("42.4831", "-71.6093", userYear)
longLat12 = addingData("42.4831", "-71.9466", userYear)
longLat13 = addingData("42.7092", "-70.9347", userYear)
longLat14 = addingData("42.7092", "-71.272", userYear)
longLat15 = addingData("42.7092", "-71.6093", userYear)
longLat16 = addingData("42.7092", "-71.9466", userYear)

# The Regions ranked

ranking= [longLat1, longLat2, longLat3, longLat4, longLat5, longLat6, longLat7, longLat8, longLat9, longLat10, longLat11, longLat12, longLat13, longLat14, longLat15, longLat16]
orange = {}
black = {}
red = {}
purple = {}
maxi = max(ranking)
mini = min(ranking)
interval = maxi - mini
interval = interval * 0.25 
for i in range(len(ranking)): 
    if ranking[i] >= mini and ranking[i] <= mini + interval: 
        orange[i] = [ranking[i], lat_long[i]]
        #orange["lat/long" + str(i)] = lat_long[i]
    if ranking[i] >= mini + interval and ranking[i] <= mini + interval + interval: 
        purple[i] = [ranking[i], lat_long[i]]
        #purple["lat/long" + str(i) ] = lat_long[i]
    if ranking[i] >= mini + interval + interval and ranking[i] <= maxi - interval: 
        red[i] = [ranking[i], lat_long[i]]
        #red["lat/long" + str(i)] = lat_long[i]
    if ranking[i] >= maxi - interval and ranking[i] <= maxi: 
        black[i] = [ranking[i], lat_long[i]]
        #black["lat/long" + str(i) ] = lat_long[i]
    else: 
        pass
    



m = Map(basemap=basemaps.Hydda.Full, center=(42.456105, -71.4466), zoom=9)
'''polygon = Polygon(
    locations=[(42.709166, -71.9466), (42.709166, -70.9347), (42.030833, -70.9347), (42.030833, -71.9466)],
    color="gray",
    fill_color="gray"
) '''





# This is a custom VectorTileLayer subclass, allowing to pass our api key to the url
class CustomVectorTileLayer(VectorTileLayer):
    api_key = Unicode('gCZXZglvRQa6sB2z7JzL1w').tag(sync=True, o=True)

water_style = dict(
fill="true",
weight=10,
fillColor="#06cccc",
color="#06cccc",
fillOpacity=0.2,
opacity=0.4,
)

waterway_style = dict(
    weight=0, fillColor="#2375e0", color="#2375e0", fillOpacity=0, opacity=0
)

admin_style = dict(
    weight=0, fillColor="pink", color="pink", fillOpacity=0, opacity=0
)

landcover_style = dict(
    fill="true",
    weight=0,
    fillColor="#2cff07",
    color="#2cff07",
    fillOpacity=0,
    opacity=0
)

landuse_style = dict(
    fill="true",
    weight=5,
    fillColor="#2cff07",
    color="#2cff07",
    fillOpacity=0.5,
    opacity=0.5,
)

park_style = dict(
    fill="true",
    weight=0,
    fillColor="#84ea5b",
    color="#84ea5b",
    fillOpacity=0,
    opacity=0,
)

boundary_style = dict(
    weight=0, fillColor="#c545d3", color="#c545d3", fillOpacity=0, opacity=0
)


aeroway = dict(
    weight=1, fillColor="#51aeb5", color="#51aeb5", fillOpacity=0.2, opacity=0.4
)

road = dict(
    weight=0, fillColor="#f2b648", color="#f2b648", fillOpacity=0, opacity=0
)

transit = dict(
    weight=0, fillColor="#f2b648", color="#f2b648", fillOpacity=0, opacity=0
)

buildings = dict(
    fill="true",
    weight=0,
    fillColor="#2b2b2b",
    color="#2b2b2b",
    fillOpacity=0.2,
    opacity=0.4,
)

water_name = dict(
    weight=0, fillColor="#022c5b", color="#022c5b", fillOpacity=0.2, opacity=0.4
)

transportation_name = dict(
    weight=0, fillColor="#bc6b38", color="#bc6b38", fillOpacity=0, opacity=0
)

place = dict(
    weight=0, fillColor="#f20e93", color="#f20e93", fillOpacity=0, opacity=0
)

housenumber = dict(
    weight=0, fillColor="#ef4c8b", color="#ef4c8b", fillOpacity=0, opacity=0
)

poi = dict(weight=0, fillColor="#3bb50a", color="#3bb50a", fillOpacity=0, opacity=0)

earth = dict(
    fill="true",
    weight=0,
    fillColor="#c0c0c0",
    color="#c0c0c0",
    fillOpacity=0.2,
    opacity=0.4,
)

url = 'https://tile.nextzen.org/tilezen/vector/v1/512/all/{z}/{x}/{y}.mvt?api_key={apiKey}'
vector_tile_layer_styles = dict(
    water=water_style,
    waterway=waterway_style,
    admin=admin_style,
    landcover=landcover_style,
    landuse=landuse_style,
    park=park_style,
    boundaries=boundary_style,
    aeroway=aeroway,
    roads=road,
    transit=transit,
    buildings=buildings,
    water_name=water_name,
    transportation_name=transportation_name,
    places=place,
    housenumber=housenumber,
    pois=poi,
    earth=earth
)

vl = CustomVectorTileLayer(url=url, vector_tile_layer_styles=vector_tile_layer_styles)
m.add_layer(vl)



for key in orange: 
    info = orange[key]
    val = info[0]
    coor = info[1]
    lat = float(coor[0])
    long = float(coor[1])
    newLat = lat + 0.113061
    newLong = long - 0.16865
    rectangle_key = Rectangle(bounds=((newLat, newLong), (newLat - 0.226122, newLong + 0.3373)), color = "black", fill_color = "orange", fill_opacity = 0.5)
    m.add_layer(rectangle_key);
for key in red: 
    info = red[key]
    val = info[0]
    coor = info[1]
    lat = float(coor[0])
    long = float(coor[1])
    newLat = lat + 0.113061
    newLong = long - 0.16865
    rectangle_key = Rectangle(bounds=((newLat, newLong), (newLat - 0.226122, newLong + 0.3373)), color = "black", fill_color = "red", fill_opacity = 0.5)
    m.add_layer(rectangle_key);
for key in black: 
    info = black[key]
    val = info[0]
    coor = info[1]
    lat = float(coor[0])
    long = float(coor[1])
    newLat = lat + 0.113061
    newLong = long - 0.16865
    rectangle_key = Rectangle(bounds=((newLat, newLong), (newLat - 0.226122, newLong + 0.3373)), color = "black", fill_color = "black", fill_opacity = 0.5)
    m.add_layer(rectangle_key);
for key in purple: 
    info = purple[key]
    val = info[0]
    coor = info[1]
    lat = float(coor[0])
    long = float(coor[1])
    newLat = lat + 0.113061
    newLong = long - 0.16865
    rectangle_key = Rectangle(bounds=((newLat, newLong), (newLat - 0.226122, newLong + 0.3373)), color = "black", fill_color = "purple", fill_opacity = 0.5)
    m.add_layer(rectangle_key);

# Creates an antpath around the area of interest 
ant_path = AntPath(
    locations=[
       [42.709166, -71.9466], [42.709166, -70.9347], [42.030833, -70.9347], [42.030833, -71.9466], [42.709166, -71.9466]
    ],
    dash_array=[15, 20],
    delay=500,
    color='#7590ba',
    pulse_color='#3f6fba'
)

m.add_layer(ant_path)

legend = LegendControl({"Low":"orange", "Mild to medium":"purple", "Medium to high":"red", "High to extremely high":"black"}, name="Color Key", position="topright")
m.add_control(legend);



m

What year would you like? Enter 2017, 2018, or 2019:      2018


Map(center=[42.456105, -71.4466], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …